Объединяю и чищу объявления WW2

In [1]:
# импорт библиотек
import pandas as pd
pd.options.mode.chained_assignment = None  # отключаю предупреждения default='warn'

import os

# прогресс-бар
from tqdm.notebook import trange, tqdm

### Объединяю файлы

In [2]:
# путь к файлам

file_path = 'C:/00_Data/temp_data/'

In [3]:
# вытаскиваю названия файлов в список file_list

file_list = []

for root, dirs, files in os.walk(file_path + 'original/'):
    
    for filename in files:
        file_list += [filename]

In [4]:
# объединяю в один датафрейм

new_work = pd.DataFrame() # новый датафрейм

for file in tqdm(file_list):
    temp = pd.read_excel(file_path + 'original/' + file)
    
    temp.drop_duplicates(subset=['topic_url'], keep='last', inplace=True)
    
    new_work = new_work.append(temp, ignore_index = True)
    
print('Количество строк: ', new_work.shape[0])

# удаляю дубликаты
new_work = new_work.drop_duplicates()

print('Количество строк после удаления дублей: ', new_work.shape[0])

  0%|          | 0/100 [00:00<?, ?it/s]

Количество строк:  191927
Количество строк после удаления дублей:  191927


In [5]:
new_work.drop_duplicates(subset=['topic_url'], keep='last', inplace=True)

In [6]:
new_work.shape

(191603, 24)

In [7]:
new_work.sample(2)

,topic_url,category,name,description,price,price_type,item_number,country,region,start_date,...,delivery,delivery_cost,payment,nickname,seller_status,seller_group,rate_pos,rate_neg,user_details,temp_dict
23213,https://forum.ww2.ru/index.php?showtopic=4918661,"ВА, Германия, Копаные, Архив",Большая ложка,Посеребренка с немецких позиций.\nПересыл на п...,Не продано,блиц цена,none,Россия,Калининград,23 янв 2021,...,Почта России,300 ₽,Оплата на карту,*******,none,Пользователи,422,-0,*******\nПользователи\n6 094 сообщений\nНа фор...,"{'Страна продавца': 'Россия', 'Расположение ло..."
8914,https://forum.ww2.ru/index.php?showtopic=4678871,"ВА, Германия, Erkennungsmarke, Архив",Контрельефный жетон Мертвой Головы,Жетон в хорошем состоянии\nВ представлении не ...,"Продано,",аукцион,none,Россия,Москва,17 июн 2017,...,Почта России,100 ₽,Безналичный расчет,*******,none,Член клуба форума,2122,-3,*******\nЧлен клуба форума\n14 138 сообщений\n...,"{'Страна продавца': 'Россия', 'Расположение ло..."


In [ ]:
# для проверки скачанных объявлений

temp_new_work = new_work.copy()
temp_new_work['category'] = temp_new_work['category'].str.replace(', Архив', '')

(temp_new_work.category.value_counts().reset_index()
 .to_excel(file_path + 'temp.xlsx', index=False))

### Добавляю статус продажи

In [8]:
# разделяю датасет на два

new_work_actual = new_work[~new_work.category.str.contains('Архив', case=True)]
new_work_archive = new_work[new_work.category.str.contains('Архив', case=True)]

In [9]:
new_work_archive.shape[0] + new_work_actual.shape[0] == new_work.shape[0]

True

In [10]:
# 2 -- для вещей, которые ещё в продаже

new_work_actual['sale_status'] = 2

In [11]:
# 1 -- для проданных вещей, 0 -- для непроданных

new_work_archive['sale_status'] = new_work_archive['price']

new_work_archive['sale_status'] = new_work_archive['sale_status'].str.replace('Снято с торгов администрацией', '0')
new_work_archive['sale_status'] = new_work_archive['sale_status'].str.replace('Не продано', '0')
new_work_archive['sale_status'] = new_work_archive['sale_status'].str.replace(r'Продано.+', '1', regex=True)
new_work_archive['sale_status'] = new_work_archive['sale_status'].str.replace(r'\d.+', '1', regex=True)

# Удаляю Продано из цен
new_work_archive['price'] = new_work_archive['price'].str.replace('Продано, ', '')

In [12]:
new_work_archive.sale_status.value_counts()

0       79761
1       53918
none    25716
Name: sale_status, dtype: int64

In [13]:
# объединяю

new_work = pd.concat([new_work_archive, new_work_actual], axis=0)

In [14]:
# убираю Архив из категорий

new_work['category'] = new_work['category'].str.replace(', Архив', '')

### Убираю лишнее

In [15]:
new_work['name'] = new_work['name'].str.replace(r'[^\w\s]+', ' ', regex=True)
new_work['name'] = new_work['name'].str.replace(r'\s+', ' ', regex=True)
new_work['name'] = new_work['name'].str.replace('ё', 'e', regex=True)

In [16]:
new_work['name'].fillna('none', inplace=True)

In [17]:
new_work.shape

(191603, 25)

In [18]:
unless_things = ['шинел', 'бурка', 'куртка', 'гимнастер', 'рубаха', 'нательное', 'белье',
                 'брюк', 'шаровар', 'штаны', 'галиф', 'бридж', 'лампасы',
                 'сапоги', 'ботинки', 'гамаши',
                 'пилотка', 'кепка', 'фуражка', 'папаха',
                 'погоны', 'воротник', 'эполет',
                 'кубик', 'кубари', 'катушки', 'шпалы', 'треугольник', 'петлицы',
                 'сюртук', 'френч', 'мундир', 'китель', 'маскостюм', 'халат', 'платье',
                 'лента', 'повязка', '',
                 'США', 'ленд лиз', 
                 'афганка', 'форма', 'униформ', 'плащ-палатка']

In [19]:
%%time

for thing in unless_things:
    new_work = (new_work[~
                         (
                             (new_work.category.str.contains('униформа', case=False)) &
                             (new_work.name.str.contains(f'{thing}', case=False))
                         )])

Wall time: 17.9 s


In [20]:
new_work.shape

(184005, 25)

In [ ]:
(new_work['name'].value_counts().reset_index()
.to_excel(file_path + 'temp.xlsx', index=False)
)

In [ ]:
new_work['name'].value_counts().reset_index().to_excel(file_path + 'temp.xlsx', index=False)

In [ ]:
new_work.sample()

In [ ]:
(new_work.price.value_counts().reset_index()
 .to_excel(file_path + 'sample.xlsx', index=False)
)

### Чищу регионы

In [38]:
new_temp = new_work.copy()

In [42]:
new_temp['region'] = new_temp['region'].str.lower()
new_temp['region'] = new_temp['region'].str.replace('г. ', '', regex=False)
new_temp['region'] = new_temp['region'].str.replace('обл.', 'область', regex=False)
new_temp['region'] = new_temp['region'].str.replace('спб', 'санкт-петербург', regex=False)

new_temp['region'] = new_temp['region'].str.replace(r'[^\w\s]+', ' ', regex=True)
new_temp['region'] = new_temp['region'].str.replace('\s+', ' ', regex=True )

new_temp['region'].fillna('none', inplace=True)

In [58]:
new_work['region'] = new_work['region'].apply(lambda x: str(x).strip())

In [59]:
new_temp.region.nunique()

1353

In [61]:
new_temp.region.value_counts().reset_index().to_excel(file_path + 'region.xlsx', index=False)